# Recurrent models

In this notebook we train several recurrent networks and evaluate their accuracy for the stated problem (classification of EEG signals).

In [2]:
%cd ..

/home/quentin/Documents/Courses/Deep learning/Projects/project1/src


In [3]:
import numpy as np

from utils.loading import load_data, save_obj, load_obj

import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

from utils.cross_validation import early_stopping
from utils.cross_validation import cross_validation

from utils.visualization import plot_history

In [4]:
one_khz = False

x_train, y_train = load_data(one_khz=one_khz)
x_test, y_test = load_data(train=False, one_khz=one_khz)

## Tests

In [17]:
from rec_models import BasicLSTM

In [18]:
model = BasicLSTM(50, 2)
history = model.fit(x_train, y_train, x_test, y_test, epochs=1, lr=5*1e-5, batch_size=16,
                    standardize=True, noise=2, crop=False)
plot_history(history)

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f83eccb9278>>
Traceback (most recent call last):
  File "/home/quentin/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/quentin/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/quentin/miniconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/quentin/miniconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/quentin/miniconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/quentin/miniconda3/lib/python3.6/multiprocessing/resource_sharer.py"

RuntimeError: size mismatch, m1: [16 x 1400], m2: [2 x 2] at /opt/conda/conda-bld/pytorch-cpu_1524582300956/work/aten/src/TH/generic/THTensorMath.c:2033